In [ ]:
import numpy as np
import scipy.special as sps
import sys,math

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
def step(q):
    k = len(q)-1
    a = np.diag((np.arange(k,dtype=float)+1)/k,1) + np.diag((k-np.arange(k,dtype=float))/k,-1) # - np.eye(k+1)
    return np.dot(a,q)

def iteratesteps(q,steps = 1):
    if steps == 1:
        return step(q)
    else:
        return iteratesteps(step(q),steps - 1)

def binomdistr(k = 5):
    return sps.binom(k,np.arange(k+1))/2**k

def deltastart(i,k = 5):
    r = np.zeros(k+1)
    r[i] = 1.
    return r

In [ ]:
q = binomdistr(4)
q = deltastart(1,6)
iteratesteps(q,10)

In [ ]:
q = deltastart(1,6)
maxsteps = 10
for i in range(maxsteps):
    plt.plot(q)
    q = step(q)
plt.plot(binomdistr(6))
plt.show()